# Module 2: Path Following and Odometry

Here we will use the wheel's angular velocities and our robot commands to create a model for our For reference, they will be printed after running the code below. (You can copy and paste the output into the next part in Module 2), and follow a specified path.

This Module should follow Module 1: Basic Motion and Calibration.

### Modeling: Part 2 <a id="Modeling2"></a>

First, let's rerecord the measurements we measured in the first module. Enter your values in the code below. (If you copied the output at the end of Module 1, you can also paste that into this block).

In [ ]:
# TODO: Replace these values with what you found in Module 1
diameter = 0.06 # meters
length = 0.12 # meters
max_speed = 44.062 # rad/s

Next, we want to create a model for how our robot moves with this information. In order to accomplish this, we must first make two assumptions to simplify our equations.

1. The wheels do not skid (move sideways).
2. The wheels never slip.

With this, we can find the equations shown below regarding the relations between the angular velocity of the wheels and the velocities of the robot in the world frame.

$$\dot{x}=\frac{r}{2}\left(\omega_{R}+\omega_{L}\right)cos\left(\theta\right)
\\[2ex]
\dot{y}=\frac{r}{2}\left(\omega_{R}+\omega_{L}\right)sin\left(\theta\right)
\\[2ex]
\dot{\theta}=\frac{r}{L}\left(\omega_{R}-\omega_{L}\right)
$$

The variables ***x*** and ***y*** refers to the coordinates in the world frame that the robot is in, while theta (***$\theta$***) refers to the orientation of the robot, or which direction it is facing. The dots over the heads indicate that these are the speeds or velocities of these values.

We are almost able to calculate the position and orientation of our robot, but it still relies on one last piece of information to calculate. In order to find our place within a frame, we need our previous location and the time since the last position was calculated. These previous positions will be denoted as $x_{n-1}$, $y_{n-1}$, and $\theta_{n-1}$. The equations are similar to generic rate equations, and are as follows:

$$x_n=\dot{x}\left(t_n-t_{n-1}\right)+x_{n-1}
\\[2ex]
y_n=\dot{y}\left(t_n-t_{n-1}\right)+y_{n-1}
\\[2ex]
\theta_n=\dot{\theta}\left(t_n-t_{n-1}\right)+\theta_{n-1}
$$

Knowing these equations, let's create a program that can keep track and update for our robot! Similar to how the <b><span style="color:#154734">JetBot</span></b> library uses Robot, we are going to implement it as a class, which will make it easier for us to store our coordinates and create functions based on this.

First, let's import our modules and initialize some objects/values.

In [ ]:
from jetbot import Robot
import time
import numpy as np
from IPython.display import display
import ipywidgets as widgets
import threading

robot = Robot()
radius = diameter/2.0
stop_event = threading.Event()

You should recognize the first two packages from Module 1, but the rest are commonly used libraries for programming that you will see throughout these modules. If you are interested in researching these further, you will see numpy is used for common mathematical functions and values, IPython and ipywidgets are for displaying values in the Jupyter Notebook, and threading is a way to run multiple code lines simultaneously (which will be important when we want to move the <b><span style="color:#154734">JetBot</span></b> and keep track of the position at the same time).

Now we want to create a class that will store our coordinates internally and allow us to update our location. The update location will be added as a method, which is basically a function specifically made for a class. It also stores (and initializes) our initial points at all zero.

In [ ]:
class Robot_Location():
    x = 0.0
    y = 0.0
    theta = 0.0
    t = None
    
    def update_pose(self):
        if self.t is None:
            self.t = time.time()
        else:
            w_R = robot.right_motor.value*max_speed
            w_L = robot.left_motor.value*max_speed

            x_dot = radius/2*(w_R + w_L)*np.cos(np.deg2rad(self.theta))
            y_dot = radius/2*(w_R + w_L)*np.sin(np.deg2rad(self.theta))
            theta_dot = np.rad2deg(radius/length*(w_R - w_L))

            t_new = time.time()
            dt = t_new - self.t
            self.x = x_dot*dt + self.x
            self.y = y_dot*dt + self.y
            self.theta = ((theta_dot*dt + self.theta) + 360) % 360
            self.t = t_new

### Path Following

Now that we have a way of determining position and orientation, we can attempt to create a path that our <b><span style="color:#154734">JetBot</span></b> follows. Our goal will be to drive a variety of directions and record the various positions. To accomplish this task, we must rely on our odometry to continuously run and record its positions, which is why we will create a function that runs with threading, then set a custom program to move the robot in various manners.

To test the performance of your odometry, we are first going to create a function that will continuously update the position of our robot. Run the code below so future blocks will recognize the function.

In [ ]:
def update_odometry():
    global robot_enabled
    global x_coords
    global y_coords
    while not stop_event.is_set():
        robot_location.update_pose()
        try:
            x_widget.value = f"X: {robot_location.x:.3f}"
            y_widget.value = f"Y: {robot_location.y:.3f}"
            theta_widget.value = f"\u03B8: {robot_location.theta:.3f}°"
        except Exception as e:
            pass
        if robot_enabled:
            x_coords.append(robot_location.x)
            y_coords.append(robot_location.y)
        time.sleep(0.1)

Now, our next cell is where we will run our program that will simultaneously move our <b><span style="color:#154734">JetBot</span></b> and record our position. But what you will need to do is add the various motions that the robot should move. Remember that there are 5 different commands you can call to move the robot.

> Move robot forward
>
> - *robot.forward(speed)*
>
> Move robot backwards
>
> - *robot.backward(speed)*
>
> Rotate left in place
>
> - *robot.left(speed)*
>
> Rotate right in place
>
> - *robot.right(speed)*
>
> Set custom speeds
>
> - *robot.set_motors(left_speed, right_speed)*

Inside the parentheses is where you should be entering your values. When putting in any values for these speeds, they should remain between **0 and 1**

You can also have the robot stop moving, or in other words, setting the motor speeds to zero.

> Stop robot from moving
>
> - *robot.stop()*

Additionally, you can also set the time that the movement should run for.

> Time delay before next running next line
>
> - *time.sleep(duration)*

To give you an example of how this would be used, the following block would move the robot forward for 1 second at 20% speed, then rotate left at 10% speed for half a second, then stop. You can also see more examples in Module 1.

> *robot.forward(0.2)*
>
> *time.sleep(1.0)*
>
> *robot.left(0.1)*
>
> *time.sleep(0.5)*
>
> robot.stop()

You are now going to use these commands to create your own path. Type your own sequence of instructions between the dashed lines, then run the block.

<div class="alert alert-block alert-danger">
<b>WARNING!</b> The <b><span style="color:#154734">JetBot</span></b> will move when you run the following code.
</div>

In [ ]:
# Widgets to display coordinates
x_widget = widgets.Label(value="X: ---")
y_widget = widgets.Label(value="Y: ---")
theta_widget = widgets.Label(value="\u03B8: ---")

display(widgets.HBox([x_widget, y_widget, theta_widget]))

x_coords = []
y_coords = []
robot_enabled = False
robot_location = Robot_Location()
thread = threading.Thread(target=update_odometry, daemon=True)
thread.start()

# Add your robot path between the two dashed lines!
robot_enabled = True
# ---------- Start Here ----------
robot.forward(0.2)
time.sleep(3.0)
robot.right(0.1)
time.sleep(1.0)
robot.backward(0.15)
time.sleep(6.0)
robot.set_motors(0.3, 0.2)
time.sleep(2.0)
# ----------- End Here -----------
robot.stop()
robot_enabled = False

stop_event.set()
thread.join()
stop_event.clear()

<div class="alert alert-block alert-info">
One good activity would be to see who can create the best star with the plot.
</div>

You should hopefully be seeing the $X$, $Y$, and $\theta$ change as it moves. But what does this path actually look like? When you run the following cell, it will create a plot of the recorded points, including a green dot to denote the starting point, and a red dot to denote the ending point. Make sure the motion in the graph matches what the robot followed in real life.

In [ ]:
import cv2
import numpy as np
from IPython.display import Image
from jetbot import bgr8_to_jpeg

# Find min and max values
min_x = int(min(x_coords)) - 1
max_x = int(max(x_coords)) + 1
min_y = int(min(y_coords)) - 1
max_y = int(max(y_coords)) + 1

# Blank Image
image_width, image_height = 720, 540
image = np.ones((image_height, image_width, 3), dtype=np.uint8) * 255  # white background

# Padding around the plot area (in pixels)
pad = 40
plot_width = image_width - 2 * pad
plot_height = image_height - 2 * pad

# Coordinate Transform
def logical_to_image_coords(x, y):
    u = int(pad + (x - min_x) / (max_x - min_x) * plot_width)
    v = int(pad + (max_y - y) / (max_y - min_y) * plot_height)
    return u, v

# Axis Location
axis_x, axis_y = logical_to_image_coords(0, 0)

# Vertical Gridlines
for gx in range(min_x, max_x + 1):
    if gx != 0:
        x, _ = logical_to_image_coords(gx, 0)
        cv2.line(image, (x, 0), (x, image_height - 1), (220, 220, 220), 1)
        cv2.putText(image, str(gx), (x - 10, axis_y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)

# Horizontal Gridlines
for gy in range(min_y, max_y + 1):
    if gy != 0:
        _, y = logical_to_image_coords(0, gy)
        cv2.line(image, (0, y), (image_width - 1, y), (220, 220, 220), 1)
        cv2.putText(image, str(gy), (axis_x + 5, y + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)

# Draw Axes
x0, y0 = logical_to_image_coords(0, min_y)
x1, y1 = logical_to_image_coords(0, max_y)
cv2.line(image, (0, axis_y), (image_width - 1, axis_y), (0, 0, 0), 2)


x0, y0 = logical_to_image_coords(min_x, 0)
x1, y1 = logical_to_image_coords(max_x, 0)
cv2.line(image, (axis_x, 0), (axis_x, image_height - 1), (0, 0, 0), 2)

for i, (x, y) in enumerate(zip(x_coords, y_coords)):
    u, v = logical_to_image_coords(x, y)
    if i == 0:
        color = (0, 255, 0)    # Green
    elif i == len(x_coords) - 1:
        color = (0, 0, 255)    # Red
    else:
        color = (255, 0, 0)    # Blue
    cv2.circle(image, (u, v), 5, color, -1)

# Save and Display
cv2.imwrite('robot_path.jpg', image)
display(Image(data=bgr8_to_jpeg(image)))

### Bonus Challenge

Let's see how well the odometry truly functions. Rather than operating our motors based on time, we will operate based on position. This works by continuously update our odometry to give feedback to our code as to where it is and stop when it reaches the desired position, creating what is known as a closed loop system.

To test and compare the performance of your odometry, place a stop sign a certain distance away (anywhere from 2-5 meters) and see how accurate it is to stopping right in front of the sign you placed. The code below will test this performance, only the distance needs to be updated based on your environment.

In [ ]:
# TODO: Update distance here
target_distance = 2.0 # meters

# Widgets to display coordinates
x_widget = widgets.Label(value="X: ---")
y_widget = widgets.Label(value="Y: ---")
theta_widget = widgets.Label(value="\u03B8: ---")

display(widgets.HBox([x_widget, y_widget, theta_widget]))

robot_location = Robot_Location()


while(True):
    robot_location.update_pose
    robot.set_motors(.2, .2)
    x_widget.value = f"X: {robot_location.x:.2f}"
    y_widget.value = f"Y: {robot_location.y:.2f}"
    theta_widget.value = f"\u03B8: {robot_location.theta:.2f}°"
    if robot_location.x >= target_distance:
        robot.stop()
        break
    time.sleep(0.02)

Hopefully, you should see that it stops right before the sign!

If you need to force it to stop, hit the square at the top to interrupt the kernel, then running the following code to make your robot stop.

In [ ]:
robot.stop()